# Aspect-Based Sentiment Analysis

Code taken from: https://medium.com/analytics-vidhya/aspect-based-sentiment-analysis-a-practical-approach-8f51029bbc4a

In [1]:
import stanza
import nltk
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer

In [2]:
stanza.download('en')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

2021-04-12 13:12:42 INFO: Downloading default packages for language: en (English)...
2021-04-12 13:12:43 INFO: File exists: /Users/TL/stanza_resources/en/default.zip.
2021-04-12 13:12:45 INFO: Finished downloading models and saved to /Users/TL/stanza_resources.
[nltk_data] Downloading package stopwords to /Users/TL/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/TL/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/TL/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/TL/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
nlp = stanza.Pipeline(lang='en', tokenize_pretokenized=True)

2021-04-12 13:12:46 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-04-12 13:12:46 INFO: Use device: cpu
2021-04-12 13:12:46 INFO: Loading: tokenize
2021-04-12 13:12:46 INFO: Loading: pos
2021-04-12 13:12:46 INFO: Loading: lemma
2021-04-12 13:12:46 INFO: Loading: depparse
2021-04-12 13:12:47 INFO: Loading: sentiment
2021-04-12 13:12:47 INFO: Loading: ner
2021-04-12 13:12:48 INFO: Done loading processors!


In [4]:
sia = SentimentIntensityAnalyzer()

In [5]:
text = "I hate the service, but the food is great!"

In [6]:
def absa(txt, nlp, stopwords, sia):
    txt = txt.lower()
    sent_list = nltk.sent_tokenize(txt)
    
    fcluster = []
    totalfeatureList = []
    finalcluster = []
    dic = {}
    
    for sent in sent_list:
        wordlist = nltk.word_tokenize(sent)
        tags = nltk.pos_tag(wordlist)
        
        # Get new tokens and pos tags after joining nouns
        new_wordlist = join_nouns(tags)
        new_tags = nltk.pos_tag(new_wordlist)
        
        # Remove stopwords
        new_tags = [i for i in new_tags if i[0] not in stopwords]
        doc = nlp([new_wordlist])
    
        # Getting the dependency relations between the words
        dep_node = []
        for dep_edge in doc.sentences[0].dependencies:
            dep_node.append([dep_edge[2].text, dep_edge[0].id, dep_edge[1]])
        
        # Convert to appropriate format
        for node in dep_node:
            if (int(node[1]) != 0):
                node[1] = new_wordlist[(int(node[1]) - 1)]
        
        featureList = []
        categories = []
        for i in new_tags:
            if(i[1]=='JJ' or i[1]=='NN' or i[1]=='JJR' or i[1]=='NNS' or i[1]=='RB'):
                featureList.append(list(i)) # For features for each sentence
                totalfeatureList.append(list(i)) # Stores the features of all the sentences in the text
                categories.append(i[0])

        for i in featureList:
            filist = []
            for j in dep_node:
                if((j[0]==i[0] or j[1]==i[0]) and (j[2] in ["nsubj", "acl:relcl", "obj", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"])):
                    if(j[0]==i[0]):
                        filist.append(j[1])
                    else:
                        filist.append(j[0])
            fcluster.append([i[0], filist])
            
    for i in totalfeatureList:
        dic[i[0]] = i[1]
    
    for i in fcluster:
        if(dic[i[0]]=="NN"):
            finalcluster.append(i)
            
    
    # Sentiment Analysis
    results = []
    for aspect, opinion_words in finalcluster:
        sentiment = sia.polarity_scores(" ".join(opinion_words))['compound']
        results.append((aspect, sentiment))
        
    return(results)

In [7]:
def join_nouns(pos_tag_list):
    """Given a list of 2-tuples (word, pos_tag),
    join all consecutive 'NN' tags to create a new sentence.
    
    Returns:
     -- final_text: the new sentence with joined nouns.
     -- new_wordlist: the tokens for the new sentence.
    """
    new_wordlist = []
    noun_buffer = []
    for idx, elem in enumerate(pos_tag_list):
        word, pos = elem
        if pos in ("NN", "NNS"):
            noun_buffer.append(word)
        else:
            if noun_buffer:
                new_wordlist.append(''.join(noun_buffer))
                noun_buffer.clear()
            new_wordlist.append(word)
        
        if idx == len(pos_tag_list)-1:
            if noun_buffer:
                new_wordlist.append(''.join(noun_buffer))
#     final_text = ' '.join(new_wordlist[:-1]) + '.'
    return new_wordlist

In [8]:
stop = stopwords.words('english')
absa(text, nlp, stop, sia)

[('service', -0.5719), ('food', 0.6249)]

In [9]:
text = """As usual, I have sequestered myself consuming no media about the Galaxy S21 and S21 Ultra. So I get to have my very own fresh reaction. And you know what? The first thing I wanna say about these is, who does Samsung think they are exactly? Like OnePlus? They've lowered the prices. But, they've also gotten rid of, or reduced, some of the things that make the Galaxy S series, the S series. This, is pretty though. Wow! That matte, like, blasted finish on the S21, looks absolutely fantastic. And like the contrast with the with like the glossy aluminum mid plate, that looks really good, doesn't it? It almost distracts you from the fact that Samsung threw so much shade at Apple, about not including chargers with their phones, and then went and didn't include a charger with their phone. I get it. You guys gotta save a buck, okay? You lowered the prices, at least to Samsung's credit. They lowered the prices of the phones. But don't pretend this is anything other than a cost saving measure. If Apple really cared about the environmental impact of providing all those chargers, what they would have done is what Xiaomi did, which is allow you to have a charger if you want or not have a charger if you don't need it and not charge you extra for it. And there's a couple other things about these devices that make them less Samsung S series for me, and more like, OnePlus. So for example, the S21 and the S21 Plus are both only available with a maximum of eight gigs of RAM. Now, it's LPDDR4X memory, it's freaking fast. But only eight gigs of memory. And only up to 256 gigs of storage. And on top of that, they actually got rid of the Micro SD slot. So on the bottom, you've got your SIM slot, you've got your speaker port, you've got your USB type C charging and data port. On the left, you've got no Bixby button, whatsoever. On the top, you've got a couple of microphones for noise cancellation. And then over on the right, you got your volume rocker and your lock button. And that is pretty much it for the physical buttons. Feel in the hand on the S21, really nice. S21 Ultra, also nice. Especially after having played around with the iPhone 12 quite a bit lately, I do appreciate rounded corners on devices that don't kind of stick into your hand. But, that's not to say that I appreciate everything here. So the S21 and S21 Plus are the first S series devices that I can remember, that actually use 1080p class displays. The Ultra has a 1440p class display, and both of them are running 120 Hertz. So you get that really nice fluidity. So nice. But, as part of the apparent cost saving measures that Samsung has gone to here, they've done away with 1440p displays. Now, in fairness, Samsung already shipped their devices by default. And in fact, even the S21 Ultra ships by default running at 1080p resolution in order to save power even though it has more physical pixels than that. But, it's just a weird move backwards to me to go to a 1080p display. Personally, I can't tell the difference at this screen size. But if you've got really sharp eyes, 1440p can look a little bit nicer. Which isn't to say though, that the displays on these phones aren't gonna look freaking awesome. The S21 and S21 Plus both max out at 1300 nits. Meaning that, like I said before with the S20, the odds are, that for people behind this phone this is gonna be the best HDR content consumption display in the building you live in, while the S20 Ultra actually maxes out at 1500 nits. I'm a little curious about the speakers. That's one of the things that tends to be a cost-saving measure when you're tryna lower the prices. Still stereo. Man, this display looks good. Actually, pretty darn loud too. Let's take something with dialogue though. I did have to change the default display profile from vivid to natural in order to make SDR content look good. But it should be noted, I didn't feel like I had to do the same thing with HDR content. So, your mileage may vary there. Remember, HDR content includes metadata in the video itself that tells the display how to behave. - So this really unfortunate thing happened between shoot days one and two of this. A big rock chipped - Really nice. Actually, both of them sound great. One area Samsung didn't cost down at all is in the SoC. So both of these are featuring Snapdragon 888s in the North America region. And then Exynos 2100s outside of that. And these are pretty sick CPUs. Both of them are built using Samsung's new five nanometer process, and both of them are using new ARM X1 cores. So just like we saw in previous generation Snapdragon SoCs, or Exynos, for that matter, you've got for low power cores for high efficiency and for high power cores for higher performance. But here instead of four lower power cores and then four of the same high power cores, we've actually got three cortex A78s which are pretty high performance, but built using ARM's traditional balance of power consumption and performance. We've also got a single X1 core. So this is ARM's attempt to kind of steal some of Apple's thunder when it comes to single threaded performance. Now, as for whether it actually gets there, now, maybe not yet. But, it's definitely a step in the right direction. Just like the news that Samsung is continuing to work with AMD on some kind of Radeon mobile graphics. It could be a huge step in the right direction. I would be very excited to see, not just the generational improvements that we're seeing in CPU, GPU, and AI performance like we are now, but like a big, like huge multi-generational leap in GPU performance. 'Cause that could be an absolute game changer for them. Now I wanna talk about another compromise. While unlike the S21, the S21 Ultra does have flagship class memory and storage configurations, going up to 16 gigs of RAM and 512 gigs of storage. Both of them have kind of mediocre charging capabilities. So we've got a 4800 milliamp hour battery here. 4000 here, this is a small upgrade. This is basically the same as last gen. Both of them max out charging over a wire at 25 Watts. Wirelessly, at just 15 Watts. And with reverse charging, that's a measly four and a half Watts. Compared to some of the Chinese phones out there, these are like, these are numbers that are like half of what's available on the market. In some cases, less than half. But I've gotta say that I appreciate Samsung's conservative approach here. As someone who cares about the longevity of my device, I actually go out of my way to charge my battery with a wire at just one amp. So that's just five Watts. And the reason that I do that is 'cause there's less risk. I mean, the last thing Samsung needs is another phone that like, lights on fire all over the place. And, it helps your battery last longer. So if you're the kind of person who keeps your phone for longer than a year or two, you should seriously consider charging at lower rates. Even if your phone does support crazy 65 watt charging or like, you know, 25, 35 watt wireless charging. Now let's run through some of the additional differences between the Ultra and the regular S21. Honestly, for me, it's at the point where these are such completely different classes of products. I don't even really understand why they share the S21 name. They've both got 5G connectivity that's included with the Snapdragon 888. But, the Ultra has Wi-Fi 6E instead of just regular Wi-Fi 6, which isn't a huge speed advantage. But, it could be a big advantage in terms of congestion. So if you live in an apartment or something like that, Wi-Fi 6E could mean that you're fighting a lot less with your neighbors. Assuming you have a Wi-Fi 6E router, and getting better Wi-Fi speeds on your mobile device. Another big difference is that while they both do have 120 Hertz displays, which gives you that sexy smoothness, only the Ultra goes down to 10 Hertz with its variable refresh rate technology. The regular S21 and the S21 Plus, even though I don't have it on hand, only go down to 48 Hertz. Which means they could end up being slightly less power efficient, depending on the application. Finally, we've got the cameras. These are two very different classes of cameras. The regular S21 has a 12 megapixel main shooter, which isn't that crazy, except that for years Samsung kind of chased the mega pixel race. And now they're going, "Oh, you know what? Maybe Apple's got a point and it's better to just capture more light as opposed to being able to capture a ton of pixels under ideal lighting situations." So the S21 has a 64 megapixel telephoto, and then a 12 megapixel ultra-wide. Moving over to the S21 Ultra, we get a honking 108 megapixel main shooter. So, the mega pixel race is back. Two 10 megapixel telephotos and then a 12 megapixel wide. And the reason for two telephotos is that they're doing some like, trippy AI combination thing for a theoretical maximum of 100 times zoom. Now I'm expecting that to be a gimmick, but who knows? Maybe I'll be pleasantly surprised. So we're gonna go regular wide angle, taking a picture of the little dude, Adam and Jake over there. I'm gonna go 3X zoom, take another picture. I'm gonna go 10X zoom. Wow, that actually looks really good. And this stabilization is like, freaking outstanding. I think 10X looks like it might be the sweet spot. It's definitely over sharpened. But that's obviously, way more detail left in the image than if we just tried to zoom in the main shooter. And this is on the Ultra. And then we're gonna step it up. Let's go 30. Oh, cool. So when you press the little preview up here, it actually locks the stabilization. Well, you can't really see it now 'cause I'm not zoomed into something. But it actually works really well. That's super cool. So you gotta kind of frame the way you want it. You just press that, and then it'll like, hold your frame. There we go. 30X is just a blurry mess. Like we could've just as easily digitally zoomed this. And a 100X is, I don't know, man. Maybe, the AI is doing a little bit there. It looks a little less over sharpened. But there's like almost no color saturation in it. Finally, there's the difference in selfie cameras. The Ultra gets a 40 megapixel selfie camera, which is like, what? And then it's just a regular 10 megapixel one on the S21 and I was S21 Plus. Oh, I can, okay. To Samsung's credit, the natural color of the selfie camera is like outstanding. Like they're not doing any of that stupid beautifying and smoothing. Or if they are, it's really not much. That looks, like really, that looks really natural. You ready for some 8K, Brandon? - [Brandon] Yeah. - I can already tell even looking at this tiny display that it's like hot garbage. - [Recording] I can already tell even looking at this tiny display that it's like, hot garbage. - Wow, the microphone's great though. Like if I was vlogging on this thing, I think I'd be stoked. It can do 4K 60. Oh man, that just like already looks better than the 8K mode. Whatever, it doesn't matter. You know what, they've got it, they gotta market (mumbles) Now for the most important test for me. The Ultra actually has support, for Samsung's S-Pen. Which isn't to say that it comes with one or even has a place to put one. In fact, you've gotta buy it as a separate accessory. I think it's like $40 US or something like that. And then you also have to get a case that has somewhere to put your S-Pen. So it's like, ah, okay. But to their credit, apparently, they've worked on intergenerational compatibility. And while some people are curious about whether the Note20 Pen works, I actually wanna take it back a couple more steps and see if my Note9 Pen will Work. Wait, it totally works. That's freaking awesome. I actually love the S-Pen, for no reason other than scribbling on screenshots and sending them to people. It's really nice or like, writing down the dimensions of a room for my trip to Ikea. Like I'll scribble on my picture. Or this one. So you can record a GIF of any video you watch, just like, going like this, going like that. And boom, you can save it, you can share it. It's like, freaking awesome. If you're a social media nerd, or whatever the case may be. I just wish there was, somewhere to put it. So that's sort of a downer. But hey, at least it works. So that's cool. It's really nice to see Samsung putting that kind of effort in. Unless it just like worked on its own, I don't know. But honestly, one of the reasons that I have decided to stick with the Note9 for so long is to see how well it does last. I've given Samsung a hard time in the past for not providing enough software updates for their devices and not supporting them in the long term. And overall, other than the couple of drops that have shattered the back glass on this thing, the experience of using it today is basically like it has always been. And I think they've done a pretty darn good job of it for me. So I'm still getting software updates. It's still not sluggish. I'm pretty happy and it's getting me to the point where I'm getting closer anyway to being able to recommend Samsung in the same way, almost, that I would Apple. Apple is still, way better. But hey, improvements improvement. Speaking of Apple, I actually told you guys on Short Circuit that I was going to do a full review of at least one of the iPhone 12 series. So the one that I've been driving is the 12 Pro. And the reason I didn't end up doing it is I just, I didn't have anything to say. So I'm gonna change gears for a little bit. And I'm gonna tell you some of the notes that I had down, if you guys actually cared. And then I'm gonna just put that to rest. So I have stopped being disingenuous about your eco-friendliness. If you actually cared about the environment, there's lots of other things you could do to help save the environment. I didn't like the hard edges. The notch is a piece of (beep) Hole punch is a piece (beep) too, but the notch notch is worse, I guess. I can't wait for that trend to go away forever. I missed 3D touch. Still, 3D touch was amazing. 60 Hertz blows. Some of the apps are better, so that's cool. Some of them are worse, so that's less cool. Face ID sucks in a pandemic 'cause I'm wearing a mask all the time. I think one of the reasons that my iPhone reviews tend to end up really nitpicky and people label me as a hater is that, it's just mostly good. It's fine, it's an iPhone. The only things I can really talk about are the things that are like, wrong. And a lot of the things that are wrong are things that have always been wrong and they just don't seem to give a (beep) about fixing, like, not adding T9 dialing. So there, that's why I didn't do it. As for whether I'm gonna review this. I don't know, what do you think? Is it worth tryna switch to? - [Jono] Yeah! - 120 Hertz display would be really nice. So smooth. I'm gonna miss my S-Pen, though. I don't know. - [Jono] Get the case! - I'm not gonna get a stupid, like case with a pen holder in it, it's ridiculous. I'm a naked phone guy. No case. If I review it, it'll be over on Linus Tech Tips but, that's it. That's it for Short Circuit for today. Oh, yeah, also they launched their smart tags. But I'm not gonna unbox a smart tag. Like what on earth would I say about it? So, good for them. Good job, Samsung. """

In [10]:
res = sorted(absa(text, nlp, stop, sia), key=lambda x: x[1], reverse=True)

In [11]:
final_res = {}
counts = {}

for aspect, polarity in res:
    # Calculate running average polarity for each aspect
    counts[aspect] = counts.get(aspect, 0) + 1
    final_res[aspect] = (final_res.get(aspect, 0) * (counts.get(aspect, 0)-1) + polarity) / (counts.get(aspect))
    final_res[aspect] = round(final_res[aspect], 2)

# Remove neutral words from final_res
final_res = {k:v for k,v in final_res.items() if abs(v) > 0.1}

In [12]:
nltk.pos_tag(["awesome"])

[('awesome', 'NN')]

In [13]:
sorted(final_res.items(), key=lambda x: x[1], reverse=True)

[('color', 0.76),
 ('hdrcontentconsumptiondisplay', 0.64),
 ('microphone', 0.62),
 ('smoothness', 0.53),
 ('buck', 0.49),
 ('power', 0.49),
 ('darnloud', 0.49),
 ('environment', 0.49),
 ('someone', 0.46),
 ('spot', 0.46),
 ('job', 0.44),
 ('feel', 0.42),
 ('fluidity', 0.42),
 ('hertzdisplay', 0.42),
 ('everything', 0.4),
 ('approach', 0.4),
 ('tag', 0.4),
 ('step', 0.36),
 ('megapixelselfiecamera', 0.36),
 ('reaction', 0.32),
 ('leap', 0.32),
 ('stabilization', 0.3),
 ('s21name', 0.3),
 ('test', 0.2),
 ('battery', 0.19),
 ('touch', 0.15),
 ('point', 0.15),
 ('phone', 0.15),
 ('apple', 0.13),
 ('thing', -0.11),
 ('shooter', -0.11),
 ('couple', -0.12),
 ('speedadvantage', -0.24),
 ('awesome', -0.42),
 ('glass', -0.48),
 ('notch', -0.48),
 ('cpus', -0.51),
 ('beautifying', -0.53)]

Todo:
* Separate the combined words.
* Average the sentiments.
* Use word embeddings to group together similar words.